In [1]:
import robotic as ry
import numpy as np
import matplotlib.pyplot as plt
import time
import sys
import open3d as o3d
from scipy.spatial import cKDTree


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
ry.version.__version__

'0.1.10'

### Part 1.1


In [3]:

# Initialize Configuration
config = ry.Config()
config.addFile("GFiles-HW3/environment.g")

# Camera Setup
camera_view = ry.CameraView(config)
cam_list = ["camera1", "camera2", "camera3", "camera4"]
point_cloud_data = np.empty((0, 3))

def transform_to_world_frame(frame_name, local_points):
    """
    Transforms local frame points to world frame coordinates.

    :param frame_name: Frame name for transformation.
    :param local_points: Points in the local frame.
    :return: Transformed points in world frame.
    """
    frame_position = config.getFrame(frame_name).getPosition()
    frame_rotation = config.getFrame(frame_name).getRotationMatrix()
    return (frame_rotation @ local_points.T).T + frame_position

def get_bounding_box(frames, margin=0.01, z_limit=0.713):
    """
    Calculate bounding box for a group of frames.

    :param frames: List of frame names.
    :param margin: Margin around the bounding box.
    :param z_limit: Minimum Z-coordinate threshold.
    :return: Bounding box limits as min and max arrays.
    """
    bbox_min = np.array([float("inf")] * 3)
    bbox_max = np.array([-float("inf")] * 3)

    for frame in frames:
        frame_obj = config.getFrame(frame)
        mesh = frame_obj.getMeshPoints()
        world_coords = transform_to_world_frame(frame, mesh)

        bbox_min = np.minimum(bbox_min, np.min(world_coords, axis=0) - margin)
        bbox_max = np.maximum(bbox_max, np.max(world_coords, axis=0) + margin)

    bbox_min[2] = max(bbox_min[2], z_limit)  # Ensure Z-min threshold
    return bbox_min, bbox_max

def filter_points_by_bbox(bbox_min, bbox_max, points):
    """
    Filters points to those inside the bounding box.

    :param bbox_min: Minimum bounding box coordinates.
    :param bbox_max: Maximum bounding box coordinates.
    :param points: Full point cloud.
    :return: Points inside the bounding box.
    """
    return points[
        (points >= bbox_min).all(axis=1) & (points <= bbox_max).all(axis=1)
    ]

def compute_point_cloud_normals(points, neighbors=30, show=False):
    """
    Computes normals for a given point cloud.

    :param points: Input 3D points.
    :param neighbors: K-nearest neighbors for normal estimation.
    :param show: If True, visualize normals.
    :return: Array of normal vectors.
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=neighbors))
    pcd.orient_normals_consistent_tangent_plane(k=neighbors)

    if show:
        pcd.paint_uniform_color([1.0, 0.0, 0.0])
        o3d.visualization.draw_geometries([pcd], point_show_normal=True)

    return np.asarray(pcd.normals)

# Merge Camera Point Clouds
for cam in cam_list:
    camera_view.setCamera(cam)
    _, depth_image = camera_view.computeImageAndDepth(config)
    cam_cloud = ry.depthImage2PointCloud(depth_image, camera_view.getFxycxy())
    cam_cloud = cam_cloud.reshape(-1, 3)
    world_cloud = transform_to_world_frame(cam, cam_cloud)
    point_cloud_data = np.vstack([point_cloud_data, world_cloud])

# Process Objects and Compute Normals
objects = {
    "soap": ["soap_0"],
    "rolling_pin": ["rolling-pin_0", "rolling-pin_1"],
    "bottle": ["bottle_0", "bottle_1", "bottle_2", "bottle_3", "bottle_4", "bottle_5", "bottle_6", "bottle_7"]
}

bounding_boxes = {}
filtered_clouds = {}
object_normals = {}

for obj, obj_frames in objects.items():
    bbox_min, bbox_max = get_bounding_box(obj_frames)
    bounding_boxes[obj] = (bbox_min, bbox_max)

    filtered_cloud = filter_points_by_bbox(bbox_min, bbox_max, point_cloud_data)
    filtered_clouds[obj] = filtered_cloud

    normals = compute_point_cloud_normals(filtered_cloud, neighbors=30, show=True)
    object_normals[obj] = normals

    # Visualize Point Cloud for Each Object
    color = [255, 0, 0] if obj == "soap" else [0, 255, 0] if obj == "rolling_pin" else [0, 0, 255]
    obj_frame = config.addFrame(f"{obj}_cloud")
    obj_frame.setPointCloud(filtered_cloud, color)


soap_normals = object_normals["soap"]
rolling_pin_normal = object_normals["rolling_pin"]
bottle_normals = object_normals["bottle"]
soap_points = filtered_clouds["soap"]
rolling_pin_points = filtered_clouds["rolling_pin"]
bottle_points = filtered_clouds["bottle"]
# Keep View Active
config.view(True)


0

In [3]:
print(soap_points.shape)
print(soap_normals.shape)

(2170, 3)
(2170, 3)


### Part 1.2 (antipodal part takes some time; therefore, optimum pairs are written in hand after found through this algorithm) 

In [3]:
def downsample_point_cloud(points, voxel_size=0.005):
    """
    Downsamples the point cloud using a voxel grid.

    :param points: (N, 3) array of point cloud data.
    :param voxel_size: Size of the voxel grid. Smaller values retain more detail.
    :return: Downsampled point cloud as a numpy array.
    """
    # Convert to Open3D point cloud
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # Apply voxel grid downsampling
    downsampled_pcd = pcd.voxel_down_sample(voxel_size=voxel_size)
    
    # Convert back to numpy array
    downsampled_points = np.asarray(downsampled_pcd.points)
    return downsampled_points

In [4]:
def antipodal_grasp_detection(points, normals, angle_threshold=np.pi * 0.9, distance_range=(0.01, 0.1), visualize=False):
    grasp_candidates = []
    tree = cKDTree(points)  # Spatial index for efficient neighbor search

    for i in range(len(points)):
        neighbors = tree.query_ball_point(points[i], r=distance_range[1])  # Neighbors within max_distance
        for j in neighbors:
            if i >= j:  # Avoid duplicate or self-pairs
                continue

            distance = np.linalg.norm(points[i] - points[j])
            if not (distance_range[0] <= distance <= distance_range[1]):
                continue

            normal1 = normals[i]
            normal2 = normals[j]
            angle = np.arccos(np.clip(np.dot(normal1, normal2), -1.0, 1.0))
            if np.abs(angle - np.pi) < angle_threshold:
                grasp_candidates.append((points[i], points[j]))

    if visualize:
        visualize_grasp_candidates(points, grasp_candidates)

    return grasp_candidates


def visualize_grasp_candidates(points, grasp_pairs):
    """
    Visualizes grasp point pairs in a 3D space.

    :param points: (N, 3) array of point cloud data.
    :param grasp_pairs: List of grasp point pairs [(point1, point2)].
    """
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.paint_uniform_color([0.5, 0.5, 0.5])  # Gray for point cloud

    lines = []
    line_colors = []
    for grasp_pair in grasp_pairs:
        lines.append([len(points), len(points) + 1])
        line_colors.append([1.0, 0.0, 0.0])  # Red for grasp lines
        pcd.points.extend(o3d.utility.Vector3dVector(np.array(grasp_pair)))

    line_set = o3d.geometry.LineSet()
    line_set.points = pcd.points
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(line_colors)

    o3d.visualization.draw_geometries([pcd, line_set], width=800, height=600)


voxel_size = 0.005  # Adjust based on your object's scale and detail required
soap_points_downsampled = downsample_point_cloud(soap_points, voxel_size)
rolling_pin_points_downsampled = downsample_point_cloud(rolling_pin_points, voxel_size)
bottle_points_downsampled = downsample_point_cloud(bottle_points, voxel_size)

# Use downsampled points in the detection function
soap_grasp_points = antipodal_grasp_detection(soap_points_downsampled, soap_normals)
rolling_pin_grasp_points = antipodal_grasp_detection(rolling_pin_points_downsampled, rolling_pin_normal)
bottle_grasp_points = antipodal_grasp_detection(bottle_points_downsampled, bottle_normals)

# Visualizing all grasp points
print(f"Soap grasp pairs: {len(soap_grasp_points)}")
print(f"Rolling pin grasp pairs: {len(rolling_pin_grasp_points)}")
print(f"Bottle grasp pairs: {len(bottle_grasp_points)}")


Soap grasp pairs: 98833
Rolling pin grasp pairs: 309044
Bottle grasp pairs: 842821


In [29]:
""" def antipodal_grasp_detection(points, normals, angle_threshold=np.pi * 0.9, distance_range=(0.01, 0.1), visualize=False):
    """
   # Detects antipodal grasp points from point cloud data and surface normals.

    #:param points: (N, 3) array of point cloud data.
    #:param normals: (N, 3) array of normal vectors corresponding to the points.
    #:param angle_threshold: Minimum angle (in radians) between normals for antipodal grasp.
    #:param distance_range: Tuple of (min_distance, max_distance) for grasp width.
    #:param visualize: Boolean, whether to visualize detected grasp points.
    #:return: List of tuples [(point1, point2)] representing pairs of grasp points.
    """
    grasp_candidates = []
    num_points = points.shape[0]

    for i in range(num_points):
        for j in range(i + 1, num_points):
            # Compute distance between points
            distance = np.linalg.norm(points[i] - points[j])
            if not (distance_range[0] <= distance <= distance_range[1]):
                continue

            # Compute angle between normals
            normal1 = normals[i]
            normal2 = normals[j]
            dot_product = np.dot(normal1, normal2)
            angle = np.arccos(np.clip(dot_product, -1.0, 1.0))  # Clip for numerical stability

            if np.abs(angle - np.pi) < angle_threshold:
                grasp_candidates.append((points[i], points[j]))

    if visualize:
        visualize_grasp_candidates(points, grasp_candidates)

    return grasp_candidates """

In [5]:
def compute_object_center(points):
    """
    Computes the center of mass of the object based on its point cloud.

    :param points: (N, 3) array of point cloud data.
    :return: (3,) array representing the center of the object.
    """
    return np.mean(points, axis=0)


In [6]:
# Compute centers of mass for soap, rolling pin, and bottle
soap_center = compute_object_center(soap_points)
rolling_pin_center = compute_object_center(rolling_pin_points)
bottle_center = compute_object_center(bottle_points)

print("Soap Center:", soap_center)
print("Rolling Pin Center:", rolling_pin_center)
print("Bottle Center:", bottle_center)

Soap Center: [-4.11669865e-05  3.99632171e-01  7.48119107e-01]
Rolling Pin Center: [0.20275032 0.39684175 0.74995909]
Bottle Center: [-0.18631975  0.34362194  0.75316998]


In [7]:
def score_grasp_candidates(grasp_pairs, object_center, optimal_grasp_width):
    """
    Scores grasp pairs based on distance to object center and optimal grasp width.

    :param grasp_pairs: List of tuples [(point1, point2)] representing grasp pairs.
    :param object_center: (3,) array of the object's center of mass.
    :param optimal_grasp_width: Ideal grasp width for the robot gripper.
    :return: Sorted list of scored grasp pairs [(score, (point1, point2))].
    """
    scored_pairs = []
    for p1, p2 in grasp_pairs:
        # Compute center proximity
        center_distance = np.linalg.norm((p1 + p2) / 2 - object_center)

        # Compute grasp width deviation
        grasp_width = np.linalg.norm(p1 - p2)
        width_deviation = abs(grasp_width - optimal_grasp_width)

        # Combine scores (lower is better)
        score = center_distance + width_deviation
        scored_pairs.append((score, (p1, p2)))

    # Sort by score (ascending, best first)
    scored_pairs.sort(key=lambda x: x[0])
    return scored_pairs


In [8]:

soap_center = compute_object_center(soap_points)
rolling_pin_center = compute_object_center(rolling_pin_points)
bottle_center = compute_object_center(bottle_points)


In [9]:

optimal_grasp_width = 0.05  

# Score grasp pairs for soap
scored_soap_grasps = score_grasp_candidates(soap_grasp_points, soap_center, optimal_grasp_width)

# Score grasp pairs for rolling pin
scored_rolling_pin_grasps = score_grasp_candidates(rolling_pin_grasp_points, rolling_pin_center, optimal_grasp_width)

# Score grasp pairs for bottle
scored_bottle_grasps = score_grasp_candidates(bottle_grasp_points, bottle_center, optimal_grasp_width)


In [22]:
top_soap_grasps = scored_soap_grasps[:300]  # Select top 100 or something
top_rolling_pin_grasps = scored_rolling_pin_grasps[:300]
top_bottle_grasps = scored_bottle_grasps[:300]

In [10]:
def visualize_grasp_points(points, grasp_pairs, top_n=100):
    """
    Visualizes top grasp point pairs in the point cloud.
    
    :param points: (N, 3) array of the full point cloud.
    :param grasp_pairs: List of scored grasp pairs [(score, (point1, point2))].
    :param top_n: Number of top grasp pairs to visualize.
    """
    # Prepare the point cloud for visualization
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    pcd.paint_uniform_color([0.5, 0.5, 0.5])  # Set point cloud to gray

    # Prepare lines for visualization
    lines = []
    line_colors = []
    line_points = points.copy()  # To hold the grasp points as well

    for i, (_, (p1, p2)) in enumerate(grasp_pairs[:top_n]):  # Take top N grasp pairs
        idx1 = len(line_points)
        idx2 = idx1 + 1
        line_points = np.vstack([line_points, p1, p2])
        lines.append([idx1, idx2])
        line_colors.append([1.0, 0.0, 0.0])  # Red color for the grasp line

    # Convert to Open3D LineSet for visualization
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(line_points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(line_colors)

    # Visualize
    o3d.visualization.draw_geometries([pcd, line_set], width=800, height=600)


In [11]:
# Visualize top 100 grasp pairs for each object
visualize_grasp_points(soap_points, scored_soap_grasps, top_n=100)
visualize_grasp_points(rolling_pin_points, scored_rolling_pin_grasps, top_n=100)
visualize_grasp_points(bottle_points, scored_bottle_grasps, top_n=100)


In [44]:
#[array[0.01993792, 0.40039222, 0.73927161]), array([-0.0200274 ,  0.40008759,  0.73932566]] best for soap
#[array[0.21628132, 0.41095367, 0.73980883]), array([0.18046731, 0.39471482, 0.73940787]] best for oklava
#[array[-0.17950454,  0.28497283,  0.74082946]), array([-0.22031921,  0.28501877,  0.74036126]] best for sıvı sıkacagı

In [4]:
p1_soap = np.array([0.01993792, 0.40039222, 0.73927161])
p2_soap = np.array([-0.0200274 ,  0.40008759,  0.73932566])
p1_oklava = np.array([0.21628132, 0.41095367, 0.73980883])
p2_oklava = np.array([0.18046731, 0.39471482, 0.73940787])
p1_sıkacak = np.array([-0.17950454,  0.28497283,  0.74082946])
p2_sıkacak = np.array([-0.22031921,  0.28501877,  0.74036126])

In [5]:
normal_vector_sıkacak = p1_sıkacak - p2_sıkacak
unit_normal_sıkacak = normal_vector_sıkacak / np.linalg.norm(normal_vector_sıkacak)

normal_vector_sabun = p1_soap - p2_soap
unit_normal_sabun = normal_vector_sabun / np.linalg.norm(normal_vector_sabun)

normal_vector_oklava = p1_oklava - p2_oklava
unit_normal_oklava = normal_vector_oklava / np.linalg.norm(normal_vector_oklava)

### BOTOP PART

In [6]:
C = ry.Config()
C.addFile("GFiles-HW3/environment.g")


In [10]:
C.view()

0

In [8]:
juicer_grasp_point1 = p1_sıkacak
juicer_grasp_point2 = p2_sıkacak


juicer_grasp_middle = (juicer_grasp_point1 + juicer_grasp_point2) / 2

juicer_gripper_point1 = juicer_grasp_middle + 0.04 * unit_normal_sıkacak
juicer_gripper_point2 = juicer_grasp_middle - 0.04 * unit_normal_sıkacak


C.addFrame("juicer_gripper_point1").setPosition(juicer_gripper_point1).setShape(ry.ST.marker, size=[0.02])
C.addFrame("juicer_gripper_point2").setPosition(juicer_gripper_point2).setShape(ry.ST.marker, size=[0.02])


juicer_sink_point = [0.6, 0.0, 0.7]  

C.addFrame("juicer_sink_place").setPosition(juicer_sink_point).setShape(ry.ST.marker, size=[0.02])

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'juicer_gripper_point1'], ry.OT.eq, [1e1])
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'juicer_gripper_point2'], ry.OT.eq, [1e1])


solver = ry.NLP_Solver()
solver.setProblem(komo.nlp())
solver.setOptions(stopTolerance=1e-2, verbose=4)
ret = solver.solve()
print(ret)

komo.view(False, "waypoints solution for juicer")

path_juicer = komo.getPath()

juicer_sink_target = [-1, -0.3, 1]

C.addFrame("juicer_sink_target").setPosition(juicer_sink_target).setShape(ry.ST.marker, size=[0.02])

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0)  
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)


komo.addObjective(
    [1.], 
    ry.FS.poseDiff,
    ['l_gripper', 'juicer_sink_target'], 
    ry.OT.eq,
    [1e1]
)

solver = ry.NLP_Solver()
solver.setProblem(komo.nlp())
solver.setOptions(stopTolerance=1e-2, verbose=4)
ret = solver.solve()
print(ret)

path2_juicer = komo.getPath()

bot = ry.BotOp(C, False)
bot.home(C)

bot.move(path_juicer, [2.0, 3.0])
while bot.getTimeToEnd() > 0:
    bot.sync(C, 0.1)

bot.gripperCloseGrasp(ry._left, 'bottle_0')  
while not bot.gripperDone(ry._left):
    bot.sync(C, 0.1)

bot.move(path2_juicer, [3.0])  
while bot.getTimeToEnd() > 0:
    bot.sync(C, 0.1)

bot.gripperMove(ry._left, width=.75, speed=.2)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)    

bot.home(C)


{ time: 0.015625, evals: 19, done: 1, feasible: 1, sos: 8.28112, f: 0, ineq: 0, eq: 0.13114 }
{ time: 0.046875, evals: 1000, done: 1, feasible: 0, sos: 21.695, f: 0, ineq: 0, eq: 3.37787 }


In [9]:
rolling_pin_grasp_point1 = p1_oklava
rolling_pin_grasp_point2 = p2_oklava

rolling_pin_grasp_middle = (rolling_pin_grasp_point1 + rolling_pin_grasp_point2) / 2

rolling_pin_gripper_point1 = rolling_pin_grasp_middle + 0.04 * unit_normal_oklava
rolling_pin_gripper_point2 = rolling_pin_grasp_middle - 0.04 * unit_normal_oklava

C.addFrame("rolling_pin_gripper_point1").setPosition(rolling_pin_gripper_point1).setShape(ry.ST.marker, size=[0.02])
C.addFrame("rolling_pin_gripper_point2").setPosition(rolling_pin_gripper_point2).setShape(ry.ST.marker, size=[0.02])

rolling_pin_sink_point = [0.6, 0.0, 0.7] 

C.addFrame("rolling_pin_sink_place").setPosition(rolling_pin_sink_point).setShape(ry.ST.marker, size=[0.02])

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'rolling_pin_gripper_point1'], ry.OT.eq, [1e1])
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'rolling_pin_gripper_point2'], ry.OT.eq, [1e1])

solver = ry.NLP_Solver()
solver.setProblem(komo.nlp())
solver.setOptions(stopTolerance=1e-2, verbose=4)
ret = solver.solve()
print(ret)

komo.view(False, "waypoints solution for rolling-pin")

path_rolling_pin = komo.getPath()

rolling_pin_sink_target = [-1, -0.3, 1]  

C.addFrame("rolling_pin_sink_target").setPosition(rolling_pin_sink_target).setShape(ry.ST.marker, size=[0.02])

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0) 
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective(
    [1.], 
    ry.FS.poseDiff,
    ['l_gripper', 'rolling_pin_sink_target'],  
    ry.OT.eq,
    [1e1]
)

solver = ry.NLP_Solver()
solver.setProblem(komo.nlp())
solver.setOptions(stopTolerance=1e-2, verbose=4)
ret = solver.solve()
print(ret)

path2_rolling_pin = komo.getPath()

bot = ry.BotOp(C, False)
bot.home(C)

bot.move(path_rolling_pin, [2.0, 3.0])
while bot.getTimeToEnd() > 0:
    bot.sync(C, 0.1)

bot.gripperCloseGrasp(ry._left, "rolling-pin_0")  
while not bot.gripperDone(ry._left):
    bot.sync(C, 0.1)

bot.move(path2_rolling_pin, [3.0])
while bot.getTimeToEnd() > 0:
    bot.sync(C, 0.1)

bot.gripperMove(ry._left, width=.75, speed=.2)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)    

bot.home(C)


{ time: 0, evals: 13, done: 1, feasible: 1, sos: 4.55128, f: 0, ineq: 0, eq: 0.140435 }
{ time: 0.09375, evals: 900, done: 1, feasible: 0, sos: 21.6954, f: 0, ineq: 0, eq: 3.37943 }


In [10]:
soap_grasp_point1 = p1_soap
soap_grasp_point2 = p2_soap

soap_grasp_middle = (soap_grasp_point1 + soap_grasp_point2) / 2

soap_gripper_point1 = soap_grasp_middle + 0.04 * unit_normal_sabun
soap_gripper_point2 = soap_grasp_middle - 0.04 * unit_normal_sabun

C.addFrame("soap_gripper_point1").setPosition(soap_gripper_point1).setShape(ry.ST.marker, size=[0.02])
C.addFrame("soap_gripper_point2").setPosition(soap_gripper_point2).setShape(ry.ST.marker, size=[0.02])

soap_sink_point = [0.6, 0.0, 0.7]  

C.addFrame("soap_sink_place").setPosition(soap_sink_point).setShape(ry.ST.marker, size=[0.02])

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(2., 1, 5., 0)
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective([1.], ry.FS.poseDiff, ['l_gripper', 'soap_gripper_point1'], ry.OT.eq, [1e1])
komo.addObjective([2.], ry.FS.poseDiff, ['l_gripper', 'soap_gripper_point2'], ry.OT.eq, [1e1])

solver = ry.NLP_Solver()
solver.setProblem(komo.nlp())
solver.setOptions(stopTolerance=1e-2, verbose=4)
ret = solver.solve()
print(ret)

komo.view(False, "waypoints solution for soap")

path_soap = komo.getPath()

soap_sink_target = [-1, -0.3, 1]  

C.addFrame("soap_sink_target").setPosition(soap_sink_target).setShape(ry.ST.marker, size=[0.02])

komo = ry.KOMO()
komo.setConfig(C, True)
komo.setTiming(1., 1, 5., 0)  
komo.addControlObjective([], 0, 1e-0)
komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)

komo.addObjective(
    [1.],  
    ry.FS.poseDiff,
    ['l_gripper', 'soap_sink_target'],  
    ry.OT.eq,
    [1e1]
)


solver = ry.NLP_Solver()
solver.setProblem(komo.nlp())
solver.setOptions(stopTolerance=1e-2, verbose=4)
ret = solver.solve()
print(ret)

path2_soap = komo.getPath()

bot = ry.BotOp(C, False)
bot.home(C)

bot.move(path_soap, [2.0, 3.0])
while bot.getTimeToEnd() > 0:
    bot.sync(C, 0.1)


bot.gripperCloseGrasp(ry._left, "soap_0")  
while not bot.gripperDone(ry._left):
    bot.sync(C, 0.1)

bot.move(path2_soap, [3.0])  
while bot.getTimeToEnd() > 0:
    bot.sync(C, 0.1)

bot.gripperMove(ry._left, width=.01, speed=.2)
while not bot.gripperDone(ry._left):
    bot.sync(C, .1)    

bot.home(C)


{ time: 0, evals: 14, done: 1, feasible: 1, sos: 5.6177, f: 0, ineq: 0, eq: 0.0544175 }
{ time: 0.109375, evals: 1000, done: 1, feasible: 0, sos: 21.6912, f: 0, ineq: 0, eq: 3.37942 }


In [11]:
del bot
